---
# --- Day 22: Monkey Map ---
---

In [41]:
import math
import numpy as np
import re
from typing import Tuple, NamedTuple, List
from tqdm.notebook import tqdm

## Load data

In [61]:
full_puzzle_data = True

In [62]:
file_suffix = "" if full_puzzle_data else "_test"
with open(f"data/day22_input{file_suffix}.txt", "r") as f:
    data = f.read().splitlines()

In [63]:
mapping = {" ": 0, ".": 1, "#": 2}
lst = []
for row in data[:-2]:
    lst.append([mapping[c] for c in list(row)])
pad = len(max(lst, key=len))
grid = np.array([i + [0]*(pad-len(i)) for i in lst])

In [64]:
password = data[-1]
instructions = re.split(r"([RL])", password)

## --- Part One ---

In [66]:
def find_step_number(path: np.ndarray, max_steps: int) -> int:
    steps = np.cumsum(path==1)
    obstacles = np.where(path==2)[0]
    steps_to_obstacle = steps[obstacles[0]] if len(obstacles) > 0 else 99999
    return np.where(steps==min(max_steps, steps_to_obstacle))[0][0] + 1 if steps_to_obstacle > 0 else 0

def make_move(pos: Tuple[int, int], facing: int, grid: np.ndarray, instruction: str) -> (Tuple[int, int], int):
    rows, cols = grid.shape
    i, j = pos
    if re.match("[0-9]+", instruction):
        n = int(instruction)
        if facing in [0, 2]:
            path = np.append(grid[i, j+1:], grid[i, :j])
            if facing == 2:
                path = path[::-1]
            steps = find_step_number(path, n) * (1-facing)      
            pos = (i, (j+steps+cols)%cols)
        elif facing in [1, 3]:
            path = np.append(grid[i+1:, j], grid[:i, j])
            if facing == 3:
                path = path[::-1]
            steps = find_step_number(path, n) * (2-facing)
            pos = ((i+steps+rows)%rows, j)
    elif instruction == "R":
        facing = (facing + 1)%4
    elif instruction == "L":
        facing = (facing + 3)%4
    return pos, facing

In [67]:
pos = (0, np.where(grid[0,:]==1)[0][0])
facing = 0
for inst in instructions:
    pos, facing = make_move(pos, facing, grid, inst)

In [68]:
final_password = 1000*(pos[0] + 1) + 4*(pos[1] + 1) + facing
print(f"The final password is {final_password}.")

The final password is 117054.


## --- Part Two ---

In [69]:
DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]
NUM_DIRECTIONS = 4

In [70]:
class CubeFace(NamedTuple):
    value: int
    rotation: int

In [71]:
CUBE_EDGES = {
    CubeFace(value=1, rotation=0): CubeFace(value=2, rotation=3),
    CubeFace(value=1, rotation=1): CubeFace(value=4, rotation=2),
    CubeFace(value=1, rotation=2): CubeFace(value=5, rotation=3),
    CubeFace(value=2, rotation=0): CubeFace(value=3, rotation=3),
    CubeFace(value=2, rotation=1): CubeFace(value=6, rotation=2),
    CubeFace(value=3, rotation=0): CubeFace(value=1, rotation=3),
    CubeFace(value=3, rotation=1): CubeFace(value=5, rotation=2),
    CubeFace(value=3, rotation=2): CubeFace(value=6, rotation=3),
    CubeFace(value=4, rotation=0): CubeFace(value=6, rotation=1),
    CubeFace(value=4, rotation=3): CubeFace(value=2, rotation=2),
    CubeFace(value=5, rotation=0): CubeFace(value=4, rotation=1),
    CubeFace(value=5, rotation=1): CubeFace(value=6, rotation=0),
}
CUBE_EDGES = CUBE_EDGES | dict((v, k) for k, v in CUBE_EDGES.items())

In [72]:
def fold_cube(grid: np.ndarray):
    """BFS to figure out which block corresponds to which number and rotation of the cube."""
    grid_res = int(math.sqrt((grid > 0).sum() // 6))
    n = len(grid) // grid_res
    m = len(grid[0]) // grid_res
    blocks = [
        (i, j)
        for i in range(n)
        for j in range(m)
        if grid[i * grid_res][j * grid_res] != 0
    ]

    block_assignments = {blocks[0]: CubeFace(1, 0)}
    q = [blocks[0]]

    while q:
        block = q.pop(0)
        face = block_assignments[block]
        (block_i, block_j) = block
        for epi, (delta_i, delta_j) in enumerate(DIRECTIONS):
            next_block = (block_i + delta_i, block_j + delta_j)
            if next_block not in blocks or next_block in block_assignments:
                continue

            ni, nj = next_block
            if not ((0 <= ni < n) and (0 <= nj < m)):
                continue

            active_edge = CubeFace(face.value, (epi + face.rotation) % NUM_DIRECTIONS)
            nm = CUBE_EDGES[active_edge]
            r = (nm.rotation - epi + 2) % NUM_DIRECTIONS

            block_assignments[next_block] = CubeFace(nm.value, r)
            q.append(next_block)

    inv_block_assignments = {v.value: k for k, v in block_assignments.items()}
    return block_assignments, inv_block_assignments, grid_res

In [73]:
def make_move_in_cube(coord: List[int], current_block: Tuple[int, int], dindex: int, grid: np.ndarray, grid_res: int,
                      block_assignments: dict, inv_block_assignments: dict, instruction: str):

    if instruction == "R":
        dindex = (dindex + 1) % NUM_DIRECTIONS
    elif instruction == "L":
        dindex = (dindex - 1) % NUM_DIRECTIONS
    else:
        instr = int(instruction)
        for _ in range(instr):
            direction = DIRECTIONS[dindex]
            next_coord = list(coord)
            next_coord[0] = next_coord[0] + direction[0]
            next_coord[1] = next_coord[1] + direction[1]
            next_block = (next_coord[0] // grid_res, next_coord[1] // grid_res)
            next_dindex = dindex

            if next_block != current_block:
                current_assignment = block_assignments[current_block]

                exit_edge = CubeFace(
                    current_assignment.value,
                    (current_assignment.rotation + dindex) % NUM_DIRECTIONS,
                )
                next_edge = CUBE_EDGES[exit_edge]

                next_block = inv_block_assignments[next_edge.value]
                next_assignment = block_assignments[next_block]

                next_dindex = (
                    next_edge.rotation - next_assignment.rotation + 2
                ) % NUM_DIRECTIONS
                block_coord = [
                    coord[0] - current_block[0] * grid_res,
                    coord[1] - current_block[1] * grid_res,
                ]

                if dindex == 0:
                    rel_coord = block_coord[0]
                elif dindex == 1:
                    rel_coord = grid_res - 1 - block_coord[1]
                elif dindex == 2:
                    rel_coord = grid_res - 1 - block_coord[0]
                else:
                    rel_coord = block_coord[1]

                if next_dindex == 0:
                    next_coord = (rel_coord, 0)
                elif next_dindex == 1:
                    next_coord = (0, grid_res - 1 - rel_coord)
                elif next_dindex == 2:
                    next_coord = (grid_res - 1 - rel_coord, grid_res - 1)
                else:
                    next_coord = (grid_res - 1, rel_coord)

                next_coord = [
                    next_block[0] * grid_res + next_coord[0],
                    next_block[1] * grid_res + next_coord[1],
                ]

            if grid[next_coord[0]][next_coord[1]] == 1:
                coord = next_coord
                current_block = next_block
                dindex = next_dindex
            elif grid[next_coord[0]][next_coord[1]] == 2:
                break
            else:
                raise Exception("Something unexpected happened!")
    return coord, current_block, dindex

In [74]:
block_assignments, inv_block_assignments, grid_res = fold_cube(grid)
blocks = sorted(block_assignments)
current_block = blocks[0]
coord, facing = [current_block[0] * grid_res, current_block[1] * grid_res], 0

In [75]:
for inst in tqdm(instructions, total=len(instructions)):
    coord, current_block, facing = make_move_in_cube(coord, current_block, facing, grid, grid_res, block_assignments, 
                                                     inv_block_assignments, inst)

  0%|          | 0/4001 [00:00<?, ?it/s]

In [76]:
1000 * (coord[0] + 1) + 4 * (coord[1] + 1) + facing

162096